<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [65]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from bs4 import BeautifulSoup
import requests
import re
from selenium import webdriver

import warnings
warnings.filterwarnings('ignore')

# 시각화 한글 처리
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font',family=font_name)

In [66]:
class Portfolio:
    
    # 종목별 PER, PBR 수집
    def DataSet(self):
        krx_list = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13')[0]
        krx_list.종목코드 = krx_list.종목코드.map('{:06d}'.format)

        per_pbr = []
        for code in tqdm(krx_list.종목코드):
            n_url = f'https://finance.naver.com/item/main.nhn?code={code}'
            html = BeautifulSoup(requests.get(n_url, headers={'User-agent': 'Mozilla/5.0'}).text, "lxml")
            try:
                group = html.find('h4', class_='h_sub sub_tit7')
                업종 = group.find('a').text
            except:
                업종 = 'N/A'
            try:
                per = html.find('em', id='_per').text
                pbr = html.find('em', id='_pbr').text
            except:
                per, pbr = 'N/A', 'N/A'
            try:
                table = html.find('table', summary='동일업종 PER 정보')
                업종per = table.find('em').text
            except:
                업종per = 'N/A'
            per_pbr.append([업종, 업종per, per, pbr])
        per_pbr_df = pd.DataFrame(per_pbr)
        per_pbr_df.columns = ['업종2', '업종PER', 'PER', 'PBR']
        
        krx_df = krx_list[['회사명','종목코드','업종']]
        df1 = pd.concat([krx_df, per_pbr_df], axis=1)
        df1.replace('N/A', np.nan, inplace=True)
        return df1
    
    # 저평가 된 기업 찾기
    def LowValue(self):
        df1 = self.DataSet()
        df1['종목코드'] = df1['종목코드'].astype('object')
        df1['업종PER'] = df1['업종PER'].astype('float')
        df1['PER'] = df1['PER'].str.replace(',','')
        df1['PER'] = df1['PER'].astype('float')
        df1['PBR'] = df1['PBR'].apply(pd.to_numeric, errors='coerce')
        df2 = df1[(df1['업종PER'] > df1['PER']) & (df1['PBR'] <=1)].reset_index(drop=True)
        return df2
    
    # 저평가기업을 대상으로 시가총액 수집
    def TotalPrice(self): 
        df3 = self.LowValue()
        df3['시가총액(억)'] = np.nan
        for i, code in tqdm(enumerate(df3.종목코드)):
            cg_url = f'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{code}&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=701'
            html = BeautifulSoup(requests.get(cg_url, headers={'User-agent': 'Mozilla/5.0'}).text, 'lxml')
            table = html.find_all("tr")[4]
            price = table.find("td", class_="r").text
            df3['시가총액(억)'][i] = price
        return df3
    
    # 저평가기업을 대상으로 거래량, 거래대금 수집
    def Transaction(self):
        chromedriver = 'C:\\Users\\dalgo\\OneDrive\\바탕 화면\\08_Investment_Portfolio\\chromedriver.exe'
        driver = webdriver.Chrome(chromedriver)
        driver.get('https://finance.naver.com/sise/sise_quant.nhn?sosok=0')

        driver.find_element_by_xpath('//*[@id="option2"]').click()
        driver.find_element_by_xpath('//*[@id="option8"]').click()
        driver.find_element_by_xpath('//*[@id="option9"]').click()
        driver.implicitly_wait(2)
        driver.find_element_by_xpath('//*[@id="option12"]').click()
        driver.find_element_by_xpath('//*[@id="option24"]').click()
        driver.find_element_by_xpath('//*[@id="contentarea_left"]/div[2]/form/div/div/div/a[1]/img').click()

        html1 = driver.page_source
        table1 = pd.read_html(html1)[1]
        table1 = table1.dropna(subset=['종목명']).drop(['N','Unnamed: 11'], axis=1).reset_index(drop=True)

        driver.implicitly_wait(2)
        driver.find_element_by_xpath('//*[@id="contentarea"]/div[3]/div/div[2]/a').click()
        html2 = driver.page_source
        table2 = pd.read_html(html2)[1]
        table2 = table2.dropna(subset=['종목명']).drop(['N','Unnamed: 11'], axis=1).reset_index(drop=True)
        
        df4 = pd.concat([table1, table2], axis=0).sort_values(by='거래량', ascending=False).reset_index(drop=True)
        return df4